In [1]:
import numpy as np
import sympy
from sympy import symbols, factorial, Function, summation, binomial, product

n, i, j, m = symbols('n, i, j, m', integer=True, real=True)

t = symbols('t')
P = Function('P')

B = summation(binomial(n, i)*(1 - t)**(n-i)*t**i*P(i), (i, 0, n))

In [12]:
P_vect = sympy.Matrix([P(i) for i in range(8)])
P_vect

Matrix([
[P(0)],
[P(1)],
[P(2)],
[P(3)],
[P(4)],
[P(5)],
[P(6)],
[P(7)]])

In [14]:
def difference_operator(P_vect):
    return sympy.Matrix([P_vect[i+1] - P_vect[i] for i in range(len(P_vect) - 1)])
difference_operator(P_vect) #vel

Matrix([
[-P(0) + P(1)],
[-P(1) + P(2)],
[-P(2) + P(3)],
[-P(3) + P(4)],
[-P(4) + P(5)],
[-P(5) + P(6)],
[-P(6) + P(7)]])

In [16]:
difference_operator(difference_operator(P_vect)) #second deriv

Matrix([
[P(0) - 2*P(1) + P(2)],
[P(1) - 2*P(2) + P(3)],
[P(2) - 2*P(3) + P(4)],
[P(3) - 2*P(4) + P(5)],
[P(4) - 2*P(5) + P(6)],
[P(5) - 2*P(6) + P(7)]])

In [17]:
difference_operator(difference_operator(difference_operator(P_vect))) #third deriv

Matrix([
[-P(0) + 3*P(1) - 3*P(2) + P(3)],
[-P(1) + 3*P(2) - 3*P(3) + P(4)],
[-P(2) + 3*P(3) - 3*P(4) + P(5)],
[-P(3) + 3*P(4) - 3*P(5) + P(6)],
[-P(4) + 3*P(5) - 3*P(6) + P(7)]])

In [2]:
# https://en.wikipedia.org/wiki/B%C3%A9zier_curve

In [3]:
B.subs({n: 5}).doit()

t**5*P(5) + 5*t**4*(1 - t)*P(4) + 10*t**3*(1 - t)**2*P(3) + 10*t**2*(1 - t)**3*P(2) + 5*t*(1 - t)**4*P(1) + (1 - t)**5*P(0)

In [8]:
C = sympy.factorial(n)/sympy.factorial(n - j)*summation((-1)**(i + j)*P(i)/(factorial(i)*factorial(j - i)), (i, 0, j))
C

factorial(n)*Sum((-1)**(i + j)*P(i)/(factorial(i)*factorial(-i + j)), (i, 0, j))/factorial(-j + n)

In [9]:
n0 = 6

P_vect = sympy.Matrix([P(i) for i in range(n0)])

C_matrix = sympy.Matrix([C.subs({j: j0, n: n0}).doit() for j0 in range(n0)])
C_matrix

A = C_matrix.jacobian(P_vect)

C_to_B = np.array(A.inv(), dtype=float)
C_to_B

C_matrix

Matrix([
[                                                    P(0)],
[                                        -6*P(0) + 6*P(1)],
[                             15*P(0) - 30*P(1) + 15*P(2)],
[                  -20*P(0) + 60*P(1) - 60*P(2) + 20*P(3)],
[         15*P(0) - 60*P(1) + 90*P(2) - 60*P(3) + 15*P(4)],
[-6*P(0) + 30*P(1) - 60*P(2) + 60*P(3) - 30*P(4) + 6*P(5)]])

In [10]:
A

Matrix([
[  1,   0,   0,   0,   0, 0],
[ -6,   6,   0,   0,   0, 0],
[ 15, -30,  15,   0,   0, 0],
[-20,  60, -60,  20,   0, 0],
[ 15, -60,  90, -60,  15, 0],
[ -6,  30, -60,  60, -30, 6]])

In [6]:
C0 = np.array([0, 0.0, 0.0, 7.824377367426159, -11.192660392995778, 4.368283025569619])
C_to_B@C0

array([0.        , 0.        , 0.        , 0.39121887, 0.81869811,
       0.90934906])

In [7]:
def de_casteljau(t, coefs):
    beta = [c for c in coefs] # values in this list are overridden
    n = len(beta)
    for j in range(1, n):
        for k in range(n - j):
            beta[k] = beta[k] * (1 - t) + beta[k + 1] * t
    return beta[0]